In [1]:
import os
import cv2
import numpy as np

#Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
folder_dir = "image/"
data = []
label = []

SIZE = 224 #Crop the image to 128x128

for folder in os.listdir(folder_dir):
    # print(folder)
    for file in os.listdir(os.path.join(folder_dir, folder)):
        print(file)
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE,SIZE))
            data.append(im)
        else:
            continue

100.jpg
12.jpg
13.jpg
16.jpg
17.jpg
2.jpg
20.jpg
21.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
63.jpg
64.jpg
65.jpg
66.jpg
67.jpg
68.jpg
69.jpg
7.jpg
70.jpg
71.jpg
72.jpg
73.jpg
74.jpg
75.jpg
76.jpg
77.jpg
78.jpg
79.jpg
8.jpg
80.jpg
81.jpg
82.jpg
83.jpg
84.jpg
85.jpg
86.jpg
87.jpg
88.jpg
89.jpg
9.jpg
90.jpg
91.jpg
92.jpg
93.jpg
94.jpg
95.jpg
96.jpg
97.jpg
98.jpg
99.jpg
1.jpg
10.jpg
100.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.jpg
51.jpg
52.jpg
53.jpg
54.jpg
55.jpg
56.jpg
57.jpg
58.jpg
59.jpg
6.jpg
60.jpg
61.jpg
62.jpg
63.jpg
64.jpg
65.jpg
66.jpg
67.jpg
68.jpg
69.jpg
7.jpg
70.jpg
71.jpg
72.jpg
73.jpg
74.jpg
75.jpg
76.jpg
77.jpg
78.jpg
79.jpg
8.

In [3]:
data_arr = np.array(data)
label_arr = np.array(label)

In [4]:
# Set labels through encoding
encoder = LabelEncoder()
y = encoder.fit_transform(label_arr)
y = to_categorical(y,2)
X = data_arr/255

In [5]:
X

array([[[[0.66666667, 0.66666667, 0.66666667],
         [0.6745098 , 0.6745098 , 0.6745098 ],
         [0.67843137, 0.67843137, 0.67843137],
         ...,
         [0.71372549, 0.71372549, 0.71372549],
         [0.71372549, 0.71372549, 0.71372549],
         [0.71372549, 0.71372549, 0.71372549]],

        [[0.67058824, 0.67058824, 0.67058824],
         [0.6745098 , 0.6745098 , 0.6745098 ],
         [0.67843137, 0.67843137, 0.67843137],
         ...,
         [0.71372549, 0.71372549, 0.71372549],
         [0.71372549, 0.71372549, 0.71372549],
         [0.71372549, 0.71372549, 0.71372549]],

        [[0.6745098 , 0.6745098 , 0.6745098 ],
         [0.67843137, 0.67843137, 0.67843137],
         [0.68235294, 0.68235294, 0.68235294],
         ...,
         [0.71372549, 0.71372549, 0.71372549],
         [0.71372549, 0.71372549, 0.71372549],
         [0.71372549, 0.71372549, 0.71372549]],

        ...,

        [[0.65490196, 0.65490196, 0.65490196],
         [0.65490196, 0.65490196, 0.65490196]

In [6]:
# Split for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((132, 224, 224, 3), (34, 224, 224, 3), (132, 2), (34, 2))

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define input shape and classes
input_shape = (224, 224, 3)
num_classes = 2  # Binary classification

# Load the pre-trained ResNet50 model (excluding top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for binary classification
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [8]:
model.fit(X_train, y_train, epochs=10, validation_data=[X_test, y_test])

Epoch 1/10
5/5 [==============================] - 27s 5s/step - loss: 6.2968 - accuracy: 0.6061 - val_loss: 0.6212 - val_accuracy: 0.9118
Epoch 2/10
5/5 [==============================] - 28s 6s/step - loss: 0.3612 - accuracy: 0.9621 - val_loss: 0.5687 - val_accuracy: 0.9118
Epoch 3/10
5/5 [==============================] - 26s 5s/step - loss: 0.1270 - accuracy: 0.9773 - val_loss: 0.1662 - val_accuracy: 0.9706
Epoch 4/10
5/5 [==============================] - 26s 5s/step - loss: 0.1849 - accuracy: 0.9848 - val_loss: 0.5109 - val_accuracy: 0.9412
Epoch 5/10
5/5 [==============================] - 30s 6s/step - loss: 0.1265 - accuracy: 0.9848 - val_loss: 0.1648 - val_accuracy: 0.9706
Epoch 6/10
5/5 [==============================] - 27s 5s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1134 - val_accuracy: 0.9412
Epoch 7/10
5/5 [==============================] - 29s 6s/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.1791 - val_accuracy: 0.9706
Epoch 8/10
5/5 [==================